# Google Analytics Customer Revenue Prediction

In this notebook I am going to use the data to predict the sum of transactions per user with machine learning. 

In [1]:
import pandas as pd
import numpy as np
import os
import json
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import datetime

## Downloading the data


In [3]:
def load_data(csv_path, nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'},
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [4]:
train_data = load_data(r"C:\Users\Nikhil\train.csv.zip")

Loaded train.csv.zip. Shape: (903653, 55)


## Let's take a look at what the data looks like.

In [6]:
train = train_data.copy()
train.shape
train.head(5)

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserSize,...,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.campaignCode,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,google + online,organic,NaN,google
4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,Chrome,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,True,(not provided),organic,NaN,google


In [7]:
train.info()
train.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 55 columns):
channelGrouping                                      903653 non-null object
date                                                 903653 non-null int64
fullVisitorId                                        903653 non-null object
sessionId                                            903653 non-null object
socialEngagementType                                 903653 non-null object
visitId                                              903653 non-null int64
visitNumber                                          903653 non-null int64
visitStartTime                                       903653 non-null int64
device.browser                                       903653 non-null object
device.browserSize                                   903653 non-null object
device.browserVersion                                903653 non-null object
device.deviceCategory                                9036

,date,visitId,visitNumber,visitStartTime
count,9.036530e+05,9.036530e+05,903653.000000,9.036530e+05
mean,2.016589e+07,1.485007e+09,2.264897,1.485007e+09
std,4.697698e+03,9.022124e+06,9.283735,9.022124e+06
min,2.016080e+07,1.470035e+09,1.000000,1.470035e+09
25%,2.016103e+07,1.477561e+09,1.000000,1.477561e+09
50%,2.017011e+07,1.483949e+09,1.000000,1.483949e+09
75%,2.017042e+07,1.492759e+09,1.000000,1.492759e+09
max,2.017080e+07,1.501657e+09,395.000000,1.501657e+09


In [8]:
train["date"] = pd.to_datetime(train["date"].astype(str), format='%Y%m%d')
train["date"].head(3)

0   2016-09-02
1   2016-09-02
2   2016-09-02
Name: date, dtype: datetime64[ns]

In [9]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
import plotly 
plotly.tools.set_credentials_file(username='natkinson1', api_key='bJw4a17SHuEaxRUIsCFP')

In [12]:
train["totals.transactionRevenue"] = train["totals.transactionRevenue"].astype('float')

In [10]:
def graph_ready(groupby_variable):

    table = train.groupby(groupby_variable, as_index=True)["totals.transactionRevenue"].agg(["size", "count", "mean"])
    table.columns = ["count", "count of non-zero revenue", "mean"]
    table = table.sort_values(by="count", ascending=False)
    table = table.reset_index()
    
    return table

devicebrowser_table = graph_ready("device.browser")
devicetype_table = graph_ready("device.deviceCategory")


trace1 = go.Bar(x=devicebrowser_table["device.browser"].head(10),y=devicebrowser_table["count"].head(10))

trace2 = go.Bar(x=devicebrowser_table["device.browser"].head(10),y=devicebrowser_table["mean"].head(10))

trace3 = go.Bar(x=devicetype_table["device.deviceCategory"].head(3), y=devicetype_table["count"].head(3))

trace4 = go.Bar(x=devicetype_table["device.deviceCategory"].head(3), y=devicetype_table["mean"].head(3))

fig = tools.make_subplots(rows=2, cols=2, subplot_titles=("Device Browser Count", "Device Browser Mean Revenue", 
                                                         "Device Type Count", "Device Type Mean Revenue"))

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 2)

fig['layout'].update(height=800, width=800, title="Revenue per Category")
py.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



In [11]:
train.to_csv('prep_data.csv')